<a href="https://colab.research.google.com/github/sangeetsaurabh/tweet_sentiment_extraction/blob/master/keras_transformer/keras_roberta_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Roberta implementation

This file is implementing Roberta transformer in Keras to predict selected text for Kaggle competition.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
data_folder = "/content/drive/My Drive/tweet-sentiment-extraction/data"
roberta_folder = "/content/drive/My Drive/tweet-sentiment-extraction/roberta/"
tmp_folder = '/tmp'

In [3]:
import shutil
shutil.copyfile('/content/drive/My Drive/tweet-sentiment-extraction/data/roberta-large-merges.txt', '/tmp/roberta-large-merges.txt')
shutil.copyfile('/content/drive/My Drive/tweet-sentiment-extraction/data/roberta-large-vocab.json', '/tmp/roberta-large-vocab.json')
shutil.copyfile('/content/drive/My Drive/tweet-sentiment-extraction/data/roberta-base-config.json', '/tmp/roberta-base-config.json')


'/tmp/roberta-base-config.json'

#### Install the right modules

In [4]:
!pip install transformers

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 17.2MB/s 
     |████████████████████████████████| 890kB 20.6MB/s 
     |████████████████████████████████| 3.0MB 25.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b399b6cde2f97b936327a9d2b146e088b1a85541b76b8301095f2e7a243d2143
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
import math
print('TF version',tf.__version__)

TF version 2.2.0


### Prepare the input

In [6]:
MAX_LEN = 96
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=tmp_folder+'/roberta-large-vocab.json', 
    merges_file=tmp_folder+'/roberta-large-merges.txt', 
    lowercase=True,
    add_prefix_space=True
)
EPOCHS = 3 # originally 3
BATCH_SIZE = 32 # originally 32
PAD_ID = 1
SEED = 88888
LABEL_SMOOTHING = 0.1
tf.random.set_seed(SEED)
np.random.seed(SEED)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv(data_folder + '/train.csv').fillna('')
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [7]:
tokenizer.get_vocab_size()

50265

#### Convert to training data based upon Roberta input requirements

In [8]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask[k,:len(enc.ids)+3] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+2] = 1
        end_tokens[k,toks[-1]+2] = 1

#### Prepare the test data as well

In [9]:
test = pd.read_csv(data_folder + '/test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask_t[k,:len(enc.ids)+3] = 1

#### Build Roberta Model

In [10]:
import pickle

def save_weights(model, dst_fn):
    weights = model.get_weights()
    with open(dst_fn, 'wb') as f:
        pickle.dump(weights, f)


def load_weights(model, weight_fn):
    with open(weight_fn, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
    return model

def loss_fn(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean(loss)
    return loss


def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained("roberta-large")
    bert_model = TFRobertaModel.from_pretrained("roberta-large",config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0])
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model

#### Performance Metrics

In [11]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [12]:
test.shape

(3534, 3)

#### Train the Roberta model

In [13]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=SEED) #originally 5 splits
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model, padded_model = build_model()
        
    #sv = tf.keras.callbacks.ModelCheckpoint(
    #    '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
    #    save_weights_only=True, mode='auto', save_freq='epoch')
    inpT = [input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]]
    targetT = [start_tokens[idxT,], end_tokens[idxT,]]
    inpV = [input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]]
    targetV = [start_tokens[idxV,], end_tokens[idxV,]]
    # sort the validation data
    shuffleV = np.int32(sorted(range(len(inpV[0])), key=lambda k: (inpV[0][k] == PAD_ID).sum(), reverse=True))
    inpV = [arr[shuffleV] for arr in inpV]
    targetV = [arr[shuffleV] for arr in targetV]
    #weight_fn = '%s-roberta-%i.h5'%(VER,fold)
    weight_fn = 'roberta.h5'
    for epoch in range(1, EPOCHS + 1):
        # sort and shuffle: We add random numbers to not have the same order in each epoch
        shuffleT = np.int32(sorted(range(len(inpT[0])), key=lambda k: (inpT[0][k] == PAD_ID).sum() + np.random.randint(-3, 3), reverse=True))
        # shuffle in batches, otherwise short batches will always come in the beginning of each epoch
        num_batches = math.ceil(len(shuffleT) / BATCH_SIZE)
        batch_inds = np.random.permutation(num_batches)
        shuffleT_ = []
        for batch_ind in batch_inds:
            shuffleT_.append(shuffleT[batch_ind * BATCH_SIZE: (batch_ind + 1) * BATCH_SIZE])
        shuffleT = np.concatenate(shuffleT_)
        # reorder the input data
        inpT = [arr[shuffleT] for arr in inpT]
        targetT = [arr[shuffleT] for arr in targetT]
        model.fit(inpT, targetT, 
            epochs=epoch, initial_epoch=epoch - 1, batch_size=BATCH_SIZE, verbose=DISPLAY, callbacks=[],
            validation_data=(inpV, targetV), shuffle=False)  # don't shuffle in `fit`
        save_weights(model, weight_fn)

    print('Loading model...')
    # model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    load_weights(model, weight_fn)

    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = padded_model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-2:b-1])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()

#########################
### FOLD 1
#########################


Some weights of the model checkpoint at roberta-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


687/687 [==============================] - 257s 374ms/step - loss: 2.9305 - activation_loss: 1.4751 - activation_1_loss: 1.4554 - val_loss: 2.5125 - val_activation_loss: 1.2711 - val_activation_1_loss: 1.2415
Epoch 2/2
687/687 [==============================] - 250s 364ms/step - loss: 2.5381 - activation_loss: 1.2897 - activation_1_loss: 1.2484 - val_loss: 2.5368 - val_activation_loss: 1.3004 - val_activation_1_loss: 1.2363
Epoch 3/3
687/687 [==============================] - 250s 364ms/step - loss: 2.4315 - activation_loss: 1.2313 - activation_1_loss: 1.2002 - val_loss: 2.5196 - val_activation_loss: 1.2905 - val_activation_1_loss: 1.2291
Loading model...
Predicting OOF...
172/172 [==============================] - 32s 186ms/step
Predicting Test...
111/111 [==============================] - 20s 183ms/step
>>>> FOLD 1 Jaccard = 0.7073552574291998

#########################
### FOLD 2
#########################


Some weights of the model checkpoint at roberta-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


688/688 [==============================] - 257s 374ms/step - loss: 3.0893 - activation_loss: 1.5588 - activation_1_loss: 1.5305 - val_loss: 2.5726 - val_activation_loss: 1.3189 - val_activation_1_loss: 1.2537
Epoch 2/2
688/688 [==============================] - 254s 369ms/step - loss: 2.5971 - activation_loss: 1.3165 - activation_1_loss: 1.2805 - val_loss: 2.5292 - val_activation_loss: 1.3090 - val_activation_1_loss: 1.2202
Epoch 3/3
688/688 [==============================] - 270s 392ms/step - loss: 2.5402 - activation_loss: 1.2929 - activation_1_loss: 1.2472 - val_loss: 2.5833 - val_activation_loss: 1.3098 - val_activation_1_loss: 1.2735
Loading model...
Predicting OOF...
172/172 [==============================] - 32s 184ms/step
Predicting Test...
111/111 [==============================] - 20s 183ms/step
>>>> FOLD 2 Jaccard = 0.7048464759687185

#########################
### FOLD 3
#########################


...

688/688 [==============================] - 262s 381ms/step - loss: 3.2386 - activation_loss: 1.6328 - activation_1_loss: 1.6058 - val_loss: 2.7409 - val_activation_loss: 1.3591 - val_activation_1_loss: 1.3818
Epoch 2/2
688/688 [==============================] - 252s 367ms/step - loss: 2.6221 - activation_loss: 1.3294 - activation_1_loss: 1.2927 - val_loss: 2.5729 - val_activation_loss: 1.3011 - val_activation_1_loss: 1.2718
Epoch 3/3
688/688 [==============================] - 259s 377ms/step - loss: 2.5097 - activation_loss: 1.2748 - activation_1_loss: 1.2349 - val_loss: 2.5125 - val_activation_loss: 1.2724 - val_activation_1_loss: 1.2401
Loading model...
Predicting OOF...
172/172 [==============================] - 31s 183ms/step
Predicting Test...
111/111 [==============================] - 20s 183ms/step
>>>> FOLD 3 Jaccard = 0.7041639892302929

#########################
### FOLD 4
#########################


Some weights of the model checkpoint at roberta-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


688/688 [==============================] - 259s 377ms/step - loss: 3.2621 - activation_loss: 1.6638 - activation_1_loss: 1.5982 - val_loss: 2.6654 - val_activation_loss: 1.3572 - val_activation_1_loss: 1.3082
Epoch 2/2
688/688 [==============================] - 253s 367ms/step - loss: 2.6200 - activation_loss: 1.3235 - activation_1_loss: 1.2965 - val_loss: 2.6080 - val_activation_loss: 1.3282 - val_activation_1_loss: 1.2798
Epoch 3/3
688/688 [==============================] - 280s 406ms/step - loss: 2.6121 - activation_loss: 1.3192 - activation_1_loss: 1.2930 - val_loss: 2.5586 - val_activation_loss: 1.3030 - val_activation_1_loss: 1.2556
Loading model...
Predicting OOF...
172/172 [==============================] - 32s 184ms/step
Predicting Test...
111/111 [==============================] - 20s 183ms/step
>>>> FOLD 4 Jaccard = 0.6899594941867637

#########################
### FOLD 5
#########################


Some weights of the model checkpoint at roberta-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


688/688 [==============================] - 284s 412ms/step - loss: 3.3256 - activation_loss: 1.6787 - activation_1_loss: 1.6469 - val_loss: 2.7357 - val_activation_loss: 1.3740 - val_activation_1_loss: 1.3617
Epoch 2/2
688/688 [==============================] - 261s 379ms/step - loss: 2.6967 - activation_loss: 1.3614 - activation_1_loss: 1.3353 - val_loss: 2.5256 - val_activation_loss: 1.2787 - val_activation_1_loss: 1.2469
Epoch 3/3
688/688 [==============================] - 270s 393ms/step - loss: 2.5657 - activation_loss: 1.3004 - activation_1_loss: 1.2654 - val_loss: 2.5299 - val_activation_loss: 1.2967 - val_activation_1_loss: 1.2333
Loading model...
Predicting OOF...
172/172 [==============================] - 32s 185ms/step
Predicting Test...
111/111 [==============================] - 20s 184ms/step
>>>> FOLD 5 Jaccard = 0.6935789611642874



#### Submission

In [14]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        #print(text1)
        #print(enc)
        st = tokenizer.decode(enc.ids[a-2:b-1])
    all.append(st)

In [15]:
test['selected_text'] = all
test[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test.sample(25)

,textID,text,sentiment,selected_text
1191,12005b65fc,Waiting for my turn on wii fit gym closed,neutral,waiting for my turn on wii fit gym closed
661,bcf13877f7,Good morning everyone,positive,good morning everyone
1968,575e4a89fe,tts ridiculously sweet of you,positive,ridiculously sweet of you
826,a0b1828b67,'Brides a la mode' pow wow first thing this morning Th...,positive,lovely
992,472c3e2c41,Getting somewhere with my first 'real' KiokuDB and catal...,neutral,getting somewhere with my first 'real' kiokudb and cata...
1922,ce71d002ec,Mommas day is may 10th! Don`t forget to do something nic...,positive,nice
1043,8db4aaef4a,watching the notebook,neutral,watching the notebook
2111,895de1648c,"really tired. and have to work the whole day tomorrow, t...",negative,depresses
1695,78d89e7c64,Yeah prbly pickin up songs for SingStar. Haven`t checke...,neutral,yeah prbly pickin up songs for singstar. haven`t checke...
2220,756d255e40,is at home with a pukey boy! Poor little baby,negative,poor little baby
